In [ ]:
# Hacky way to schedule. Here I'm setting these to sleep until the gpus should be free.
# At the end of the notebooks  os._exit(00) will kill the kernel freeing the gpu. 
#                          Hours to wait
# import time; time.sleep( 6 * (60*60))

# G only KEGG based network architecture

> 

In [ ]:
import os, re, json

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"

# import hilbertcurve
# from hilbertcurve.hilbertcurve import HilbertCurve

from EnvDL.core import * # includes remove_matching_files
from EnvDL.dna import *
from EnvDL.dlfn import *

from tqdm import tqdm

In [ ]:
from graphviz import Digraph
import torchviz

In [ ]:
cache_path = '../nbs_artifacts/01.25_g2fc_demo_G_ACGT_graph_backup_multilayer_nodes/'
ensure_dir_path_exists(dir_path = cache_path)

In [ ]:
dataloader_batch_size = 1000
# dataloader_batch_size = 8 #16 #64
# run_epochs = 200

use_gpu_num = 0

# Imports --------------------------------------------------------------------
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch import nn

device = "cuda" if torch.cuda.is_available() else "cpu"
if use_gpu_num in [0, 1]: 
    torch.cuda.set_device(use_gpu_num)
print(f"Using {device} device")

Using cuda device


## Simple case: Building a Neural Net from an arbitrary graph

In [ ]:
# start by finding the top level -- all those keys which are theselves not values
# helper function to get all keys and all value from a dict. Useful for when keys don't have unique values.
def find_uniq_keys_values(input_dict):
    all_keys = list(input_dict.keys())
    all_values = []
    for e in all_keys:
        all_values.extend(input_dict[e])
    all_values = list(set(all_values))

    return({'all_keys': all_keys,
           'all_values': all_values})

### Find order that nodes in the graph should be called to have all dependencies run when they are called.

In [ ]:
# find the dependancies for run order from many dependancies to none
# wrapper function to find the nodes that aren't any other nodes dependancies.
def find_top_nodes(all_key_value_dict):
    return([e for e in all_key_value_dict['all_keys'] if e not in all_key_value_dict['all_values']])
# wrapper function to find the input nodes. They don't occur in the keys and thus won't be added to the list otherwise.
# another way to do this would have been to 
def find_input_nodes(all_key_value_dict):
    return([e for e in all_key_value_dict['all_values'] if e not in all_key_value_dict['all_keys']])

### Set up output tensor

### Set up dict of input tensors

### Figure out expected input/output shapes

_==NOTE! This assumes only dense connections!==_

### Set up DataSet

Now we have

- A dictionary with the connections: `example_dict`
- The expected input sizes for each node: `example_dict_input_size`
- A dictionary with the input tensors: `input_tensor_dict`
- A list of the input tensors' names: `no_dependants` 
- A list with the order that each module should be called: `dependancy_order`

### Set up NeuralNetwork

## Tiny test

### Tiny `n`  gene version of the network:

In [ ]:
# vis_dot_bool = True

In [ ]:
# kegg_connections = {
#     'c1':['b1'],
#     'c2':['b2'],
#     'b1':['a1_input', 'b2'],
#     'b2':['a2_input']}

In [ ]:
# # add yhat node to the graph
# temp_values = []
# for key in kegg_connections.keys():
#     temp_values += kegg_connections[key]

# kegg_connections['y_hat'] = [key for key in kegg_connections.keys() if key not in temp_values]

In [ ]:
# # This is too big to render in full
# dot = ''
# if vis_dot_bool:
#     dot = Digraph()
#     for key in tqdm(kegg_connections.keys()):
#         dot.node(key)
#         for value in kegg_connections[key]:
#             # edge takes a head/tail whereas edges takes name pairs concatednated (A, B -> AB)in a list
#             dot.edge(value, key)    

# dot

Version with the node names masked for size 

### Setup to build the graph

In [ ]:
# # start by finding the top level -- all those keys which are theselves not values
# res = find_uniq_keys_values(input_dict = kegg_connections)
# all_keys = res['all_keys']
# all_values = res['all_values']

# # use the keys to find the input/outputs of the graph
# output_nodes = [e for e in all_keys if e not in all_values]
# input_nodes = [e for e in all_values if e not in all_keys]

# # (output_nodes, input_nodes)

In [ ]:
# # find the dependancies for run order from many dependancies to none
# temp = kegg_connections.copy()

# no_dependants = find_input_nodes(all_key_value_dict = find_uniq_keys_values(input_dict = temp))
# # first pass. Same as the output nodes identified above
# dependancy_order = []
# # Then iterate
# for ith in range(100): #TODO <- this should be set as a input parameter
#     top_nodes = find_top_nodes(all_key_value_dict = find_uniq_keys_values(input_dict = temp))
#     if top_nodes == []:
#         break
#     else:
#         dependancy_order += top_nodes    
#         # remove nodes from the graph that are at the 'top' level and haven't already been removed
#         for key in [e for e in dependancy_order if e in temp.keys()]:
#              temp.pop(key)

# # dependancy_order

In [ ]:
# # reverse to get the order that the nodes should be called
# dependancy_order.reverse()
# # dependancy_order

In [ ]:
# # Trying out new approach: add a node for the input data tha will only flatten the input.
# dependancy_order = input_nodes+dependancy_order

# for key in input_nodes:
#     kegg_connections[key] = [] #[key] # needs to contain itself so the model's `get_input_node()` function works 
#                                # or that function needs to change.
    

-### Set up dict of input tensors

In [ ]:
# y_true = torch.from_numpy(np.concatenate([
#         np.ones((10, )),
#         np.zeros((10, ))], 0))

In [ ]:
# input_tensor_dict = {
#     'a1_input': torch.from_numpy(np.concatenate([
#         np.zeros((10, 4, 3)),
#         np.ones((10, 4, 3))], 0)),
#     'a2_input': torch.from_numpy(np.concatenate([
#         np.zeros((10, 4, 2)),  
#         np.ones((10, 4, 2))], 0))}

# x_list_temp = [input_tensor_dict[key].to(torch.float) for key in input_tensor_dict.keys()]

-### Figure out expected input/output shapes

_==NOTE! This assumes only dense connections!==_

In [ ]:
# # This could be replaced by a sort of "distance from output" measure
# default_output_size = 5
# output_size_dict = dict(zip(dependancy_order, 
#                         [default_output_size for i in range(len(dependancy_order))]))
# output_size_dict['y_hat'] = 1 
# # output_size_dict

In [ ]:
# # input nodes will only flatten the output so they have length channels*length.
# for key in input_nodes:
#     output_size_dict[key] = np.prod(np.array(input_tensor_dict[key].shape[1:]))

In [ ]:
# output_size_dict

In [ ]:
# # CHANNEL AWARE VERSION -----------------------------------------------------------------------------------
# input_size_dict = kegg_connections.copy()

# # use the expected output sizes from `output_size_dict` to fill in the non-data sizes
# tensor_ndim = len(input_tensor_dict[list(input_tensor_dict.keys())[0]].shape)
# for e in tqdm(input_size_dict.keys()):
#     # overwrite named connections with the output size of those connections
#     # if the entry is in no_dependants it's data so it's size needs to be grabbed from the input_tensor_dict
    
#     # is there no channel dim? (major/minor allele)
#     if 2 == tensor_ndim:
#         input_size_dict[e] = [
#             list(input_tensor_dict[ee].shape)[-1] # <- NOTE! THIS ASSUMES ONLY DENSE CONNECTIONS (i.e. only the 1st dim is needed)
#             if ee in no_dependants
#             else output_size_dict[ee] for ee in input_size_dict[e]]
#     elif 3 == tensor_ndim: # There is a channel dim
#         input_size_dict[e] = [
#             (list(input_tensor_dict[ee].shape)[1]*list(input_tensor_dict[ee].shape)[2]) # <- NOTE! THIS ASSUMES ONLY DENSE CONNECTIONS (i.e. only the 1st dim is needed)  
#             if ee in no_dependants
#             else output_size_dict[ee] for ee in input_size_dict[e]]

# # Now walk over entries and overwrite with the sum of the inputs
# for e in tqdm(input_size_dict.keys()):
#     input_size_dict[e] = np.sum(input_size_dict[e])

In [ ]:
# dot = ''
# if vis_dot_bool:
#     dot = Digraph()
#     for key in tqdm(kegg_connections.keys()):
#         key_label = str(key)+'\nin: '+str(input_size_dict[key])+'\nout: '+str(output_size_dict[key])
#         dot.node(key, key_label)
#         for value in kegg_connections[key]:
#             # edge takes a head/tail whereas edges takes name pairs concatednated (A, B -> AB)in a list
#             dot.edge(value, key)    

# dot

### Set up DataSet

Now we have

- A dictionary with the connections: `example_dict`
- The expected input sizes for each node: `example_dict_input_size`
- A dictionary with the input tensors: `input_tensor_dict`
- A list of the input tensors' names: `no_dependants` 
- A list with the order that each module should be called: `dependancy_order`

In [ ]:
class ListDataset(Dataset): # for any G containing matix with many (phno) to one (geno)
    def __init__(self, 
                 y, 
                 x_list,
                 transform = None, target_transform = None,
                 **kwargs 
                ):
        self.device = device
        self.y = y 
        self.x_list = x_list
        self.transform = transform
        self.target_transform = target_transform    
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        y_idx =self.y[idx]
        
        x_idx =[x[idx, ] for x in self.x_list] 

        
        if self.target_transform:
            y_idx = self.transform(y_idx)
            x_idx = [self.transform(x) for x in x_idx]
            
        return y_idx, x_idx
    

To have a fair test of whether the model is working, I want to ensure there is information to learn in the dataset. To this end I'm using just two genotypes.

In [ ]:
# training_dataloader = DataLoader(ListDataset(
#         y = y_true[:, None].to(torch.float32),
#         x_list = [e.to(torch.float32) for e in x_list_temp]
#     ),
#     batch_size = 2,
#     shuffle = True
# )

# # next(iter(training_dataloader))

### Set up NeuralNetwork

In [ ]:
# dot

In [ ]:
# Working version ====
# Doesn't pass output node through relu
class NeuralNetwork(nn.Module):
    def __init__(self, 
                 example_dict, # contains the node (excluding input tensors)
                 example_dict_input_size, # contains the input sizes (including the tensors)
                 example_dict_output_size,
                 input_tensor_names,
                 dependancy_order
                ):
        super(NeuralNetwork, self).__init__()
        def Linear_block(in_size, out_size, drop_pr):
            block = nn.Sequential(
                nn.Linear(in_size, out_size),
                nn.ReLU(),
                nn.Dropout(drop_pr))
            return(block)   
        
        
        # fill in the list in dependancy order. 
        layer_list = []
        for key in dependancy_order:
            if key in input_tensor_names:
                layer_list += [
                    nn.Flatten()
                ]
            elif key != 'y_hat':
                layer_list += [
                    Linear_block(in_size=example_dict_input_size[key], 
                                 out_size=example_dict_output_size[key], 
                                 drop_pr=0)
                              ]
            else:
                layer_list += [
                    nn.Linear(example_dict_input_size[key], 
                              example_dict_output_size[key])
                              ]
                

        self.nn_layer_list = nn.ModuleList(layer_list)

        # things for get_input_node in forward to work.
        self.example_dict = example_dict
        self.input_tensor_names = input_tensor_names
        self.dependancy_order = dependancy_order
        
        self.input_tensor_lookup = dict(zip(input_tensor_names, 
                                            [i for i in range(len(input_tensor_names))]))
        self.result_list = []
        self.result_list_lookup = {}
            

    def forward(self, x):
        # Note: x will be a list. input_tensor_lookup will contain the name: list index pairs.
        # I use a dict instead of a list comprehension here because there could be an arbitrarily
        # large number of inputs in the list. 
        def get_input_node(self, input_node, get_x):  
#             print(input_node, self.result_list_lookup)
            return(self.result_list[self.result_list_lookup[input_node]])
        
        # trying reinstantiating to get around inplace replacement issue.
        self.result_list = []
        self.result_list_lookup = {}
        for key in self.dependancy_order:
            input_nodes = self.example_dict[key]
            nn_layer_list_idx = [i for i in range(len(dependancy_order)) if dependancy_order[i]==key][0]
            
            self.result_list_lookup[key] = len(self.result_list_lookup)                
            if key in self.input_tensor_names: # If the input node is an input (flatten) layer
                self.result_list = self.result_list + [self.nn_layer_list[nn_layer_list_idx](
                    x[self.input_tensor_lookup[key]]
                ).clone()]

            else:
                self.result_list = self.result_list + [self.nn_layer_list[nn_layer_list_idx](torch.concat(
                    [get_input_node(self, input_node = e, get_x = x) for e in input_nodes], 
                    -1)).clone()]

        return self.result_list[self.result_list_lookup['y_hat']]
    

# model = NeuralNetwork(example_dict = kegg_connections, 
#                       example_dict_input_size = input_size_dict,
#                       example_dict_output_size = output_size_dict,
#                       input_tensor_names = list(input_tensor_dict.keys()),
#                       dependancy_order = dependancy_order) 

# model(next(iter(training_dataloader))[1])

In [ ]:
# # model = model.to(device)
# model(next(iter(training_dataloader))[1])

In [ ]:
# the only 
def train_loop_yx(dataloader, model, loss_fn, optimizer, silent = False):
#     import numpy as np
#     import pandas as pd
    import torch
    from torch.utils.data import Dataset
    from torch.utils.data import DataLoader
#     from torch import nn
    size = len(dataloader.dataset)
    for batch, (y_i, xs_i) in enumerate(dataloader):
#         print(batch)
        # Compute prediction and loss
        pred = model(xs_i)
                
        loss = loss_fn(pred, y_i) # <----------------------------------------

        # Backpropagation
        torch.autograd.set_detect_anomaly(True)
        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(y_i) # <----------------
            if not silent:
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
# train_loop_yx(dataloader = training_dataloader, 
#               model = model, 
#               loss_fn = nn.MSELoss(), 
#               optimizer = torch.optim.SGD(model.parameters(), lr=0.003),
#               silent = False)

In [ ]:
# dot

### Train loop functions

In [ ]:
def train_error_yx(dataloader, model, loss_fn, silent = False):
#     import numpy as np
#     import pandas as pd
    import torch
    from torch.utils.data import Dataset
    from torch.utils.data import DataLoader
#     from torch import nn
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss = 0

    with torch.no_grad():
        for y_i, xs_i in dataloader:
            pred = model(xs_i)
            
            # ensure both are on cuda
#             if pred.device.type == 'cpu':
#                 pred = pred.to('cuda')
#             if y_i.device.type == 'cpu':
#                 y_i = y_i.to('cuda')
            
            train_loss += loss_fn(pred, y_i).item() # <----------------------
            
    train_loss /= num_batches
    return(train_loss)

In [ ]:
def test_loop_yx(dataloader, model, loss_fn, silent = False):
#     import numpy as np
#     import pandas as pd
    import torch
    from torch.utils.data import Dataset
    from torch.utils.data import DataLoader
#     from torch import nn

    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0

    with torch.no_grad():
        for y_i, xs_i in dataloader:
            pred = model(xs_i)
            
            # ensure both are on cuda
#             if pred.device.type == 'cpu':
#                 pred = pred.to('cuda')
#             if y_i.device.type == 'cpu':
#                 y_i = y_i.to('cuda')
                
            test_loss += loss_fn(pred, y_i).item() # <-----------------------

    test_loss /= num_batches
    if not silent:
        print(f"Test Error: Avg loss: {test_loss:>8f}")
    return(test_loss)

In [ ]:
def train_nn_yx(
    cache_path,
    training_dataloader,
    testing_dataloader,
    model,
    learning_rate = 1e-3,
    batch_size = 64,
    epochs = 500,
    model_prefix = 'model',
    save_pt = False
):
    import numpy as np
    import pandas as pd
    import torch
#     from torch.utils.data import Dataset
#     from torch.utils.data import DataLoader
    from torch import nn
    from tqdm import tqdm
    
    # Initialize the loss function
    loss_fn = nn.MSELoss()

    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    loss_df = pd.DataFrame([i for i in range(epochs)], columns = ['Epoch'])
    loss_df['TrainMSE'] = np.nan
    loss_df['TestMSE']  = np.nan

    for t in tqdm(range(epochs)):        
#         print(f"Epoch {t+1}\n-------------------------------")
        train_loop_yx(training_dataloader, model, loss_fn, optimizer, silent = True)

        loss_df.loc[loss_df.index == t, 'TrainMSE'
                   ] = train_error_yx(training_dataloader, model, loss_fn, silent = True)
        
        loss_df.loc[loss_df.index == t, 'TestMSE'
                   ] = test_loop_yx(testing_dataloader, model, loss_fn, silent = True)
        
        if (t+1)%5 == 0: # Cache in case training is interupted. 
            # print(loss_df.loc[loss_df.index == t, ['TrainMSE', 'TestMSE']])
            if save_pt:
                torch.save(model.state_dict(), 
                           cache_path+'/'+model_prefix+'_'+str(t)+'_'+str(epochs)+'.pt') # convention is to use .pt or .pth
        
    return([model, loss_df])

### Fit Model

# End

## KEGG graph 

## Load data

In [ ]:
load_from = '../nbs_artifacts/01.03_g2fc_prep_matrices/'
# phno = pd.read_csv(load_from+'phno.csv')
phno_geno = pd.read_csv(load_from+'phno_geno.csv')
phno = phno_geno

obs_geno_lookup = np.load(load_from+'obs_geno_lookup.npy') # Phno_Idx	Geno_Idx	Is_Phno_Idx
YMat = np.load(load_from+'YMat.npy')
# GMat = np.load(load_from+'GMat.npy')
# ACGT_OneHot = np.load(load_from+'ACGT_OneHot.npy')
# ACGT = np.load(load_from+'ACGT.npy')
# ACGT_hilb = np.load(load_from+'ACGT_hilb.npy')
# SMat = np.load(load_from+'SMat3.npy')
# WMat = np.load(load_from+'WMat3.npy')
# MMat = np.load(load_from+'MMat3.npy')

In [ ]:
# import pickle as pkl
# with open('../data/zma/kegg/kegg_gene_entries.pkl', 'rb') as handle:
#     parsed_kegg_gene_entries = pkl.load(handle)

# Testing using the cleaned up version in '../nbs_artifacts/01.05_g2fc_demo_model/'
load_from = '../nbs_artifacts/01.05_g2fc_demo_model/'
parsed_kegg_gene_entries = get_cached_result(load_from+'filtered_kegg_gene_entries.pkl')

ACGT_gene_slice_list = get_cached_result(load_from+'ACGT_gene_slice_list.pkl')

In [ ]:
assert len(ACGT_gene_slice_list) == len(parsed_kegg_gene_entries)

In [ ]:
# Restrict to only those with pathway
kegg_gene_brite = [e for e in parsed_kegg_gene_entries if 'BRITE' in e.keys()]

In [ ]:
# also require to have a non-empty path
kegg_gene_brite = [e for e in kegg_gene_brite if not e['BRITE']['BRITE_PATHS'] == []]

In [ ]:
print('Retaining '+ str(round(len(kegg_gene_brite)/len(parsed_kegg_gene_entries), 4)*100)+'%, '+str(len(kegg_gene_brite)
     )+'/'+str(len(parsed_kegg_gene_entries)
     )+' Entries'
     )

Retaining 43.53%, 6067/13939 Entries


In [ ]:
kegg_gene_brite[1]['BRITE']['BRITE_PATHS']

[['KEGG Orthology (KO) [BR:zma00001]',
  '09100 Metabolism',
  '09102 Energy metabolism',
  '00190 Oxidative phosphorylation',
  '100383860'],
 ['Enzymes [BR:zma01000]',
  '7. Translocases',
  '7.1  Catalysing the translocation of hydrons',
  '7.1.2  Linked to the hydrolysis of a nucleoside triphosphate',
  '7.1.2.1  P-type H+-exporting transporter',
  '100383860']]

### Tiny `n`  gene version of the network:

In [ ]:
n_genes = 200
n_genes = 6067 

print('Using '+str(n_genes)+'/'+str(len(kegg_gene_brite))+' genes.')

# if n_genes is too big, don't visualize.
vis_dot_bool = True
if n_genes > 10:
    vis_dot_bool = False

Using 6067/6067 genes.


In [ ]:
# """
# The goal here is to have a dict with each node and a list of it's children. 
# For example, the graph
# a--b--d
#  |-c--e
# Would be parsed into     
# {'a':['b', 'c'],
#  'b':['d'],
#  'c':['e']}
# """
# kegg_connections = {}

# # for all genes in list
# for i in tqdm(range(n_genes)): # <------------------------- only using 2 inputs genes
# # for i in tqdm(range(len(kegg_gene_brite))):    
#     temp = kegg_gene_brite[i]['BRITE']['BRITE_PATHS']
#     # clean up to make sure that there are no ":" characters. These can mess up graphviz
#     temp = [[temp[j][i].replace(':', '-') for i in range(len(temp[j])) ] for j in range(len(temp))]
#     # all paths through graph associated with a gene
#     for j in range(len(temp)):
#         # steps of the path through the graph
#         for k in range(len(temp[j])-1):
            
            
            
#             # if this is a new key, add it and add the k+1 entry as it's child
#             if temp[j][k] not in kegg_connections.keys():
#                 kegg_connections[temp[j][k]] = [temp[j][k+1]]
#             else: 
#                 # Check to see if there's a new child to add   
#                 if temp[j][k+1] not in kegg_connections[temp[j][k]]:
#                     # make sure that no key contains itself. This was a problem for 'Others' which is now disallowed.
#                     if (temp[j][k] != temp[j][k+1]):
# #                         if ((temp[j][k] != temp[j][k+1]) & (temp[j][k+1] != 'Others')):
#                         # add it.
#                         kegg_connections[temp[j][k]].extend([temp[j][k+1]])

# # kegg_connections

In [ ]:
"""
The goal here is to have a dict with each node and a list of it's children. 
For example, the graph
a--b--d
 |-c--e
Would be parsed into     
{'a':['b', 'c'],
 'b':['d'],
 'c':['e']}
"""
kegg_connections = {}

# for all genes in list
for i in tqdm(range(n_genes)): # <------------------------- only using 2 inputs genes
# for i in tqdm(range(len(kegg_gene_brite))):    
    temp = kegg_gene_brite[i]['BRITE']['BRITE_PATHS']
    # clean up to make sure that there are no ":" characters. These can mess up graphviz
    temp = [[temp[j][i].replace(':', '-') for i in range(len(temp[j])) ] for j in range(len(temp))]
    # all paths through graph associated with a gene
    for j in range(len(temp)):
        # steps of the path through the graph
        for k in range(len(temp[j])-1):
            
            # name standardization 
            temp_jk  = temp[j][k]
            temp_jk1 = temp[j][k+1]
            temp_jk  = temp_jk.lower().title().replace(' ', '')
            temp_jk1 = temp_jk1.lower().title().replace(' ', '')
            
            # if this is a new key, add it and add the k+1 entry as it's child
            if temp_jk  not in kegg_connections.keys():
                kegg_connections[temp_jk] = [temp_jk1]
            else: 
                # Check to see if there's a new child to add   
                if temp_jk1 not in kegg_connections[temp_jk]:
                    # make sure that no key contains itself. This was a problem for 'Others' which is now disallowed.
                    if (temp_jk != temp_jk1):
#                         if ((temp_jk  != temp_jk1) & (temp_jk1 != 'Others')):
                        # add it.
                        kegg_connections[temp_jk].extend([temp_jk1])

# kegg_connections                        

100%|████████████████████████████████████| 6067/6067 [00:00<00:00, 50273.11it/s]


In [ ]:
if 'Others' in kegg_connections.keys():
    del kegg_connections['Others']
    print('Removed node "Others"')

# remove 'Others' as a possible value
for key in kegg_connections.keys():
    kegg_connections[key] = [e for e in kegg_connections[key] if e != 'Others']


Removed node "Others"


In [ ]:
# Make sure that no list contains it's own key
for key in kegg_connections.keys():
    kegg_connections[key] = [e for e in kegg_connections[key] if e != key]

In [ ]:
# [key for key in input_size_dict.keys() if type(input_size_dict[key]) != np.int64]
# input_size_dict['Other DSBR factors']

# kegg_connections['Other DSBR factors']

# start with those that are not connected to any SNPs. Remove them.

# rm_list = [key for key in kegg_connections.keys() if kegg_connections[key]==[]]

# there might be associations with no dependants and with no dependants except those that have no dependants.
# Build up a list with those keys that don't connect back to snps then I'll pass over the connection dict once to remove references to them.
rm_list = []
rm_list_i = len(rm_list)
rm_list_j = -1
for i in range(100):
    if rm_list_i == rm_list_j:
        break
    else:
        rm_list = [key for key in kegg_connections.keys() if [e for e in kegg_connections[key] if e not in rm_list]
     ==[]]
        rm_list_j = rm_list_i 
        rm_list_i = len(rm_list)
rm_list

['GProtein-CoupledReceptors[Br-Zma04030]']

In [ ]:
for key in rm_list:
    del kegg_connections[key]
    
for key in kegg_connections.keys():
    kegg_connections[key] = [e for e in kegg_connections[key] if e not in rm_list]

In [ ]:
# add yhat node to the graph
temp_values = []
for key in kegg_connections.keys():
    temp_values += kegg_connections[key]

kegg_connections['y_hat'] = [key for key in kegg_connections.keys() if key not in temp_values]

In [ ]:
# This is too big to render in full
dot = ''
if vis_dot_bool:
    dot = Digraph()
    for key in tqdm(kegg_connections.keys()):
        dot.node(key)
        for value in kegg_connections[key]:
            # edge takes a head/tail whereas edges takes name pairs concatednated (A, B -> AB)in a list
            dot.edge(value, key)    

dot

''

Version with the node names masked for size 

In [ ]:
dot = ''
if vis_dot_bool:
    name_to_num_dict = dict(zip(list(kegg_connections.keys()),
                                [str(i) for i in range(len(list(kegg_connections.keys())))]))

    temp = {}
    for key in kegg_connections.keys():
        temp[name_to_num_dict[key]] = [name_to_num_dict[e] if e in name_to_num_dict.keys() else e for e in kegg_connections[key]]

    dot = Digraph()
    for key in tqdm(temp.keys()):
        dot.node(key)
        for value in temp[key]:
            # edge takes a head/tail whereas edges takes name pairs concatednated (A, B -> AB)in a list
            dot.edge(value, key)    

    # dot.render(directory=cache_path, view=True) 
dot

''

### Setup to build the graph

In [ ]:
# start by finding the top level -- all those keys which are theselves not values
res = find_uniq_keys_values(input_dict = kegg_connections)
all_keys = res['all_keys']
all_values = res['all_values']

# use the keys to find the input/outputs of the graph
output_nodes = [e for e in all_keys if e not in all_values]
input_nodes = [e for e in all_values if e not in all_keys]

# (output_nodes, input_nodes)

In [ ]:
# find the dependancies for run order from many dependancies to none
temp = kegg_connections.copy()

no_dependants = find_input_nodes(all_key_value_dict = find_uniq_keys_values(input_dict = temp))
# first pass. Same as the output nodes identified above
dependancy_order = []
# Then iterate
for ith in range(100): #TODO <- this should be set as a input parameter
    top_nodes = find_top_nodes(all_key_value_dict = find_uniq_keys_values(input_dict = temp))
    if top_nodes == []:
        break
    else:
        dependancy_order += top_nodes    
        # remove nodes from the graph that are at the 'top' level and haven't already been removed
        for key in [e for e in dependancy_order if e in temp.keys()]:
             temp.pop(key)

# dependancy_order

In [ ]:
# reverse to get the order that the nodes should be called
dependancy_order.reverse()
# dependancy_order

In [ ]:
# Trying out new approach: add a node for the input data tha will only flatten the input.
dependancy_order = input_nodes+dependancy_order

for key in input_nodes:
    kegg_connections[key] = [] #[key] # needs to contain itself so the model's `get_input_node()` function works 
                               # or that function needs to change.

-### Set up dict of input tensors

In [ ]:
# build a dict to go from the node names in `no_dependants` to the list index in `ACGT_gene_slice_list`
brite_node_to_list_idx_dict = {}
for i in tqdm(range(len(kegg_gene_brite))):
    brite_node_to_list_idx_dict[str(kegg_gene_brite[i]['BRITE']['BRITE_PATHS'][0][-1])] = i

100%|███████████████████████████████████| 6067/6067 [00:00<00:00, 963749.52it/s]


In [ ]:
input_tensor_dict = {}
for e in no_dependants:
    input_tensor_dict[e] = ACGT_gene_slice_list[brite_node_to_list_idx_dict[e]]
    
# input_tensor_dict

-### Figure out expected input/output shapes

_==NOTE! This assumes only dense connections!==_

In [ ]:
# This could be replaced by a sort of "distance from output" measure
default_output_size = 50
output_size_dict = dict(zip(dependancy_order, 
                        [default_output_size for i in range(len(dependancy_order))]))
output_size_dict['y_hat'] = 1 
# output_size_dict

In [ ]:
# CHANNEL AWARE VERSION -----------------------------------------------------------------------------------
input_size_dict = kegg_connections.copy()

# use the expected output sizes from `output_size_dict` to fill in the non-data sizes
tensor_ndim = len(input_tensor_dict[list(input_tensor_dict.keys())[0]].shape)
for e in tqdm(input_size_dict.keys()):
    # overwrite named connections with the output size of those connections
    # if the entry is in no_dependants it's data so it's size needs to be grabbed from the input_tensor_dict
    
    # is there no channel dim? (major/minor allele)
    if 2 == tensor_ndim:
        input_size_dict[e] = [
            list(input_tensor_dict[ee].shape)[-1] # <- NOTE! THIS ASSUMES ONLY DENSE CONNECTIONS (i.e. only the 1st dim is needed)
            if ee in no_dependants
            else output_size_dict[ee] for ee in input_size_dict[e]]
    elif 3 == tensor_ndim: # There is a channel dim
        input_size_dict[e] = [
            (list(input_tensor_dict[ee].shape)[1]*list(input_tensor_dict[ee].shape)[2]) # <- NOTE! THIS ASSUMES ONLY DENSE CONNECTIONS (i.e. only the 1st dim is needed)  
            if ee in no_dependants
            else output_size_dict[ee] for ee in input_size_dict[e]]

# Now walk over entries and overwrite with the sum of the inputs
for e in tqdm(input_size_dict.keys()):
    input_size_dict[e] = np.sum(input_size_dict[e])

100%|███████████████████████████████████| 8868/8868 [00:00<00:00, 335974.71it/s]


In [ ]:
dot = ''
if vis_dot_bool:
    dot = Digraph()
    for key in tqdm(kegg_connections.keys()):
        key_label = 'in: '+str(input_size_dict[key])+'\nout: '+str(output_size_dict[key])
        dot.node(key, key_label)
        for value in kegg_connections[key]:
            # edge takes a head/tail whereas edges takes name pairs concatednated (A, B -> AB)in a list
            dot.edge(value, key)    

dot

''

### Set up DataSet

Now we have

- A dictionary with the connections: `example_dict`
- The expected input sizes for each node: `example_dict_input_size`
- A dictionary with the input tensors: `input_tensor_dict`
- A list of the input tensors' names: `no_dependants` 
- A list with the order that each module should be called: `dependancy_order`

In [ ]:
class ListDataset(Dataset): # for any G containing matix with many (phno) to one (geno)
    def __init__(self, 
                 y, 
                 x_list,
                 obs_idxs, # this is a list of the indexes used. It allows us to pass in smaller 
                           # tensors and then get the right genotype
                 obs_geno_lookup,
                 transform = None, target_transform = None,
                 use_gpu_num = 0,
                 device = 'cuda',
                 **kwargs 
                ):
        self.device = device
        self.y = y 
        self.x_list = x_list
        self.obs_idxs = obs_idxs
        self.obs_geno_lookup = obs_geno_lookup
        self.transform = transform
        self.target_transform = target_transform    
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        y_idx =self.y[idx]
        
        idx_geno = obs_geno_lookup[self.obs_idxs[idx], 1]
#         print(idx)
#         print(self.obs_idxs[idx])
#         print(obs_geno_lookup[self.obs_idxs[idx], 1])
# #         print([x.shape for x in self.x_list[0]])
#         print([e.shape for e in self.x_list])
#         print()
#         x_idx =[x[idx_geno, ] for x in self.x_list[0]] # must be the 0th entry because it's in a tuple
        x_idx =[x[idx_geno, ] for x in self.x_list] 
        
        if self.device == 'cuda':
            y_idx.to(self.device)
            x_idx = [x.to(self.device) for x in x_idx]

        if self.target_transform:
            y_idx = self.transform(y_idx)
            x_idx = [self.transform(x) for x in x_idx]
            
        return y_idx, x_idx
    

To have a fair test of whether the model is working, I want to ensure there is information to learn in the dataset. To this end I'm using just two genotypes.

In [ ]:
# # These are the genotypes with the most observations. 
# yield_xbar_summary = pd.DataFrame(zip(
#     [i for i in range(4898)],
#     [len(obs_geno_lookup[obs_geno_lookup[:, 1] == i, 0]) for i in range(4898)],
#     [np.mean(YMat[obs_geno_lookup[obs_geno_lookup[:, 1] == i, 0]]) for i in range(4898)]
# )).sort_values(2)
# yield_xbar_summary

In [ ]:
# # limit by min number of observations
# yield_xbar_summary = yield_xbar_summary.loc[(yield_xbar_summary[1] > 300), ].reset_index(drop = True)
# yield_xbar_summary = yield_xbar_summary.loc[(
#     (yield_xbar_summary.index == 0) | 
#     (yield_xbar_summary.index == np.max(yield_xbar_summary.index))), ].reset_index(drop = True)
# yield_xbar_summary

In [ ]:
# small_yield_genotype_idxs = list(obs_geno_lookup[obs_geno_lookup[:, 1] == yield_xbar_summary[0][0], 0])
# large_yield_genotype_idxs = list(obs_geno_lookup[obs_geno_lookup[:, 1] == yield_xbar_summary[1][0], 0])

In [ ]:
# # use genotypes with the biggest difference between them (provided both groups have over a 300 observations)
# idxs_two_genotypes = small_yield_genotype_idxs+large_yield_genotype_idxs

# np.random.shuffle(idxs_two_genotypes)

# test_idx = idxs_two_genotypes[0:round(len(idxs_two_genotypes)*.1)]
# train_idx = idxs_two_genotypes[round(len(idxs_two_genotypes)*.1):]

In [ ]:
## Create train/test validate indicies from json
load_from = '../nbs_artifacts/01.06_g2fc_cluster_genotypes/'

def read_json(json_path):
    with open(json_path, 'r') as fp:
        dat = json.load(fp)
    return(dat)

def read_split_info(
    load_from = '../nbs_artifacts/01.06_g2fc_cluster_genotypes/',
    json_prefix = '2023:9:5:12:8:26'):
    jsons = [e for e in os.listdir(load_from) if re.match('^'+json_prefix+'.+\.json$', e)]
    vals = [e for e in jsons if re.match('.+val\d+\.json$', e)]
    vals.sort()
    out = {}
    out['test'] = [read_json(json_path = load_from+json_prefix+'-test.json')]
    out['test_file'] = [json_prefix+'-test.json']
    out['validate'] = [read_json(json_path = load_from+val) for val in vals]
    out['validate_files'] = [val for val in vals]
    return(out)


def find_idxs_split_dict(
    obs_df, # assumes presence of Year, Female, Male
    split_dict # from read_split_info() output. Should be a test of validate dict.
):

    temp = obs_df
    test_mask = ((temp.Year.isin(split_dict['test_years'])) & 
                 ((temp.Female.isin(split_dict['test_parents'])) |
                  (temp.Male.isin(split_dict['test_parents']))))
    temp['Split'] = ''
    temp.loc[test_mask, 'Split'] = 'Test'

    train_mask = (~(temp.Year.isin(split_dict['test_years'])) & 
                 (~((temp.Female.isin(split_dict['test_parents'])) |
                  (temp.Male.isin(split_dict['test_parents'])))))
    temp.loc[train_mask, 'Split'] = 'Train'

    temp_test  = (temp.Split == 'Test') # should be the same as with the mask above
    temp_train = (temp.Split == 'Train') # should be the same as with the mask above

    # Confirm that there's no overlap in parents or years
    temp_test_parents  = set(temp.loc[temp_test, 'Female']+temp.loc[temp_test, 'Male'])
    temp_train_parents = set(temp.loc[temp_train, 'Female']+temp.loc[temp_train, 'Male'])

    temp_test_years  = set(temp.loc[temp_test, 'Year'])
    temp_train_years = set(temp.loc[temp_train, 'Year'])

    assert [] == [e for e in temp_test_parents if e in temp_train_parents]
    assert [] == [e for e in temp_train_parents if e in temp_test_parents]
    assert [] == [e for e in temp_test_years if e in temp_train_years]
    assert [] == [e for e in temp_train_years if e in temp_test_years]

    return({
        'test_idx': temp.loc[test_mask, ].index, 
        'train_idx': temp.loc[train_mask, ].index} )

split_info = read_split_info(
    load_from = '../nbs_artifacts/01.06_g2fc_cluster_genotypes/',
    json_prefix = '2023:9:5:12:8:26')

temp = phno.copy()
temp[['Female', 'Male']] = temp['Hybrid'].str.split('/', expand = True)

test_dict = find_idxs_split_dict(
    obs_df = temp, 
    split_dict = split_info['test'][0]
)
# test_dict

# since this is applying predefined model structure no need for validation.
# This is included for my future reference when validation is needed.
temp = temp.loc[test_dict['train_idx'], ] # restrict before re-aplying

val_dict = find_idxs_split_dict(
    obs_df = temp, 
    split_dict = split_info['validate'][0]
)
# val_dict

test_dict

train_idx = test_dict['train_idx']
test_idx  = test_dict['test_idx']

In [ ]:
x_list_temp = [torch.from_numpy(input_tensor_dict[key]).to(torch.float) for key in input_tensor_dict.keys()]
[e.shape for e in x_list_temp]
y_temp = torch.from_numpy(YMat).to(torch.float)#[:, None]
# FIXME

In [ ]:
x_list_temp[0].shape

torch.Size([4926, 4, 13])

In [ ]:
training_dataloader = DataLoader(ListDataset(
        y = y_temp[train_idx][:, None],
        x_list = [e for e in x_list_temp],
        obs_idxs = train_idx, 
        obs_geno_lookup = obs_geno_lookup, 
        use_gpu_num = 0,
        device = 'cuda',
    ),
    batch_size = dataloader_batch_size,
    shuffle = True
)

testing_dataloader = DataLoader(ListDataset(
        y = y_temp[test_idx][:, None],
        x_list = [e for e in x_list_temp],
        obs_idxs = test_idx, 
        obs_geno_lookup = obs_geno_lookup,
        use_gpu_num = 0,
        device = 'cuda',
    ),
    batch_size = dataloader_batch_size,
    shuffle = True
)

# next(iter(training_dataloader))[1]

### Set up NeuralNetwork

In [ ]:
# kegg_connections
# input_size_dict
# output_size_dict
# list(input_tensor_dict.keys())
# dependancy_order

In [ ]:
# Working version ====
# Doesn't pass output node through relu
class NeuralNetwork(nn.Module):
    def __init__(self, 
                 example_dict, # contains the node (excluding input tensors)
                 example_dict_input_size, # contains the input sizes (including the tensors)
                 example_dict_output_size,
                 input_tensor_names,
                 dependancy_order
                ):
        super(NeuralNetwork, self).__init__()
        def Linear_block(in_size, out_size, drop_pr):
            block = nn.Sequential(
                nn.Linear(in_size, out_size),
                nn.ReLU(),
                nn.Dropout(drop_pr))
            return(block)   
        
        
        # fill in the list in dependancy order. 
        layer_list = []
        for key in dependancy_order:
            if key in input_tensor_names:
                layer_list += [
                    nn.Flatten()
                ]
            elif key != 'y_hat':
                layer_list += [
                    Linear_block(in_size=example_dict_input_size[key], 
                                 out_size=example_dict_output_size[key], 
                                 drop_pr=0)
                              ]
            else:
                layer_list += [
                    nn.Linear(example_dict_input_size[key], 
                              example_dict_output_size[key])
                              ]
                

        self.nn_layer_list = nn.ModuleList(layer_list)

        # things for get_input_node in forward to work.
        self.example_dict = example_dict
        self.input_tensor_names = input_tensor_names
        self.dependancy_order = dependancy_order
        
        self.input_tensor_lookup = dict(zip(input_tensor_names, 
                                            [i for i in range(len(input_tensor_names))]))
        self.result_list = []
        self.result_list_lookup = {}
            

    def forward(self, x):
        # Note: x will be a list. input_tensor_lookup will contain the name: list index pairs.
        # I use a dict instead of a list comprehension here because there could be an arbitrarily
        # large number of inputs in the list. 
        def get_input_node(self, input_node, get_x):  
#             print(input_node, self.result_list_lookup)
            return(self.result_list[self.result_list_lookup[input_node]])
        
        # trying reinstantiating to get around inplace replacement issue.
        self.result_list = []
        self.result_list_lookup = {}
        for key in self.dependancy_order:
            input_nodes = self.example_dict[key]
            nn_layer_list_idx = [i for i in range(len(dependancy_order)) if dependancy_order[i]==key][0]
            
            self.result_list_lookup[key] = len(self.result_list_lookup)                
            if key in self.input_tensor_names: # If the input node is an input (flatten) layer
                self.result_list = self.result_list + [self.nn_layer_list[nn_layer_list_idx](
                    x[self.input_tensor_lookup[key]]
                ).clone()]

            else:
                self.result_list = self.result_list + [self.nn_layer_list[nn_layer_list_idx](torch.concat(
                    [get_input_node(self, input_node = e, get_x = x) for e in input_nodes], 
                    -1)).clone()]

        return self.result_list[self.result_list_lookup['y_hat']]
    

model = NeuralNetwork(example_dict = kegg_connections, 
                      example_dict_input_size = input_size_dict,
                      example_dict_output_size = output_size_dict,
                      input_tensor_names = list(input_tensor_dict.keys()),
                      dependancy_order = dependancy_order) 
model = model.to(device)
# model(next(iter(training_dataloader))[1])

### Train loop functions

In [ ]:
# the only 
def train_loop_yx(dataloader, model, loss_fn, optimizer, silent = False):
#     import numpy as np
#     import pandas as pd
    import torch
    from torch.utils.data import Dataset
    from torch.utils.data import DataLoader
#     from torch import nn
    size = len(dataloader.dataset)
    for batch, (y_i, xs_i) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(xs_i)
        
        # ensure both are on cuda
        if pred.device.type == 'cpu':
            pred = pred.to('cuda')
        if y_i.device.type == 'cpu':
            y_i = y_i.to('cuda')
        
        loss = loss_fn(pred, y_i) # <----------------------------------------

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(y_i) # <----------------
            if not silent:
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
# train_loop_yx(dataloader = training_dataloader, 
#               model = model, 
#               loss_fn = nn.MSELoss(), 
#               optimizer = torch.optim.SGD(model.parameters(), lr=0.003),
#               silent = False)

In [ ]:
def train_error_yx(dataloader, model, loss_fn, silent = False):
#     import numpy as np
#     import pandas as pd
    import torch
    from torch.utils.data import Dataset
    from torch.utils.data import DataLoader
#     from torch import nn
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss = 0

    with torch.no_grad():
        for y_i, xs_i in dataloader:
            pred = model(xs_i)
            
            # ensure both are on cuda
            if pred.device.type == 'cpu':
                pred = pred.to('cuda')
            if y_i.device.type == 'cpu':
                y_i = y_i.to('cuda')
            
            train_loss += loss_fn(pred, y_i).item() # <----------------------
            
    train_loss /= num_batches
    return(train_loss)

In [ ]:
def test_loop_yx(dataloader, model, loss_fn, silent = False):
#     import numpy as np
#     import pandas as pd
    import torch
    from torch.utils.data import Dataset
    from torch.utils.data import DataLoader
#     from torch import nn

    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0

    with torch.no_grad():
        for y_i, xs_i in dataloader:
            pred = model(xs_i)
            
            # ensure both are on cuda
            if pred.device.type == 'cpu':
                pred = pred.to('cuda')
            if y_i.device.type == 'cpu':
                y_i = y_i.to('cuda')
                
            test_loss += loss_fn(pred, y_i).item() # <-----------------------

    test_loss /= num_batches
    if not silent:
        print(f"Test Error: Avg loss: {test_loss:>8f}")
    return(test_loss)

In [ ]:
def train_nn_yx(
    cache_path,
    training_dataloader,
    testing_dataloader,
    model,
    learning_rate = 1e-3,
    batch_size = 64,
    epochs = 500,
    model_prefix = 'model',
    save_model = False
):
    import numpy as np
    import pandas as pd
    import torch
#     from torch.utils.data import Dataset
#     from torch.utils.data import DataLoader
    from torch import nn
    from tqdm import tqdm
    
    # Initialize the loss function
    loss_fn = nn.MSELoss()

    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

    loss_df = pd.DataFrame([i for i in range(epochs)], columns = ['Epoch'])
    loss_df['TrainMSE'] = np.nan
    loss_df['TestMSE']  = np.nan

    for t in tqdm(range(epochs)):        
#         print(f"Epoch {t+1}\n-------------------------------")
        train_loop_yx(training_dataloader, model, loss_fn, optimizer, silent = True)

        loss_df.loc[loss_df.index == t, 'TrainMSE'
                   ] = train_error_yx(training_dataloader, model, loss_fn, silent = True)
        
        loss_df.loc[loss_df.index == t, 'TestMSE'
                   ] = test_loop_yx(testing_dataloader, model, loss_fn, silent = True)
        
        if (t+1)%5 == 0: # Cache in case training is interupted. 
            # print(loss_df.loc[loss_df.index == t, ['TrainMSE', 'TestMSE']])
            if save_model:
                torch.save(model.state_dict(), 
                           cache_path+'/'+model_prefix+'_'+str(t)+'_'+str(epochs)+'.pt') # convention is to use .pt or .pth
        
    return([model, loss_df])

### Fit Model

In [ ]:
# ee = next(iter(training_dataloader))
# [e.shape for e in ee[1]]

In [ ]:
# dataloader_batch_size = 1000
run_epochs = 1
epochs_run = 0

model, loss_df = train_nn_yx(
    cache_path,
    training_dataloader,
    testing_dataloader,
    model,
    learning_rate = 1e-3,
    batch_size = dataloader_batch_size,
    epochs = (run_epochs - epochs_run)
)

  0%|                                                   | 0/1 [2:49:27<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
# model.to('cpu')
# model(next(iter(training_dataloader)))

In [ ]:
loss_df

px.line(loss_df, x = 'Epoch', y = 'TrainMSE')

In [ ]:
naieve_yhat = training_dataloader.dataset.y.mean()

naieve_MSE_Train = np.array(((naieve_yhat - training_dataloader.dataset.y)**2)).mean(),

naieve_MSE_Test = np.array(((naieve_yhat - testing_dataloader.dataset.y)**2)).mean(),

(naieve_MSE_Train, naieve_MSE_Test)

In [ ]:

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=loss_df.Epoch, y=loss_df.TestMSE,
                    mode='lines', name='Test'))
fig.add_trace(go.Scatter(x=loss_df.Epoch, y=loss_df.TrainMSE,
                    mode='lines', name='Train'))

fig.add_trace(go.Scatter(x=loss_df.Epoch, y=[list(naieve_MSE_Test)[0]  for e in range(len(loss_df.Epoch))], 
                         mode='lines', name='Naieve Test'))
fig.add_trace(go.Scatter(x=loss_df.Epoch, y=[list(naieve_MSE_Train)[0] for e in range(len(loss_df.Epoch))], 
                         mode='lines', name='Naieve Train'))
fig.show()

In [ ]:
stophere

In [ ]:
epochs_run = 0

In [ ]:
dataloader_batch_size = 500
run_epochs = 100

# don't run if either of these exist because there may be cases where we want the results but not the model
import re

if not os.path.exists(cache_path+'/model.pt'): 
    # Shared setup (train from scratch and load latest)
#     model = NeuralNetwork()

#     # find the biggest model to save
#     saved_models = os.listdir(cache_path)
#     saved_models = [e for e in saved_models if re.match('model*', e)]

#     if saved_models == []:
#         epochs_run = 0
#     else:
#         saved_models = [e for e in saved_models if e != 'model.pt']
#         # if there are saved models reload and resume training
#         saved_models_numbers = [int(e.replace('model_', ''
#                                     ).replace('.pt', ''
#                                     ).split('_')[0]) for e in saved_models]
#         # saved_models
#         epochs_run = max(saved_models_numbers)+1 # add 1 to account for 0 index
#         latest_model = [e for e in saved_models if re.match(
#             '^model_'+str(epochs_run-1)+'_.*\.pt$', e)][0] # subtract 1 to convert back
#         model.load_state_dict(torch.load(cache_path+latest_model))
#         print('Resuming Training: '+str(epochs_run)+'/'+str(run_epochs)+' epochs run.')
    
    model.to(device)    

    model, loss_df = train_nn(
        cache_path,
        training_dataloader,
        testing_dataloader,
        model,
        learning_rate = 1e-3,
        batch_size = dataloader_batch_size,
        epochs = (run_epochs - epochs_run)
    )
    
    # experimental outputs:
    # 1. Model
    torch.save(model.state_dict(), cache_path+'/model.pt') # convention is to use .pt or .pth

    # 2. loss_df    
    # If this is resuming training, load and extend the existing loss dataframe
    if os.path.exists(cache_path+'/loss_df.csv'):
        loss_df_on_disk = pd.read_csv(cache_path+'/loss_df.csv')
        epoch_offset = 1 + loss_df_on_disk['Epoch'].max()
        loss_df['Epoch'] = loss_df['Epoch'] + epoch_offset
        loss_df = pd.concat([loss_df_on_disk, loss_df])
    loss_df.to_csv(cache_path+'/loss_df.csv', index=False)  
    
    # 3. predictions 
    yhats = pd.concat([
        yhat_loop(testing_dataloader, model).assign(Split = 'Test'),
        yhat_loop(training_dataloader, model).assign(Split = 'Train')], axis = 0)

    yhats.to_csv(cache_path+'/yhats.csv', index=False)

In [ ]:
stophere

## One Hot Encoded

In [ ]:
rng = np.random.default_rng(9230473) # note, must use rng.shuffle(arr) below for this to take effect.

In [ ]:
pct_cumsum_thresh = .9

# make a df to aid in creating train/test splits
# the plan is to shuffle the rows of the df, calculate the cumulative sum of the percents obs, then 
# the entries above and below a given percent will be the train/test.
obs_per_hybrid = phno.assign(n = 1).groupby('Hybrid').count().reset_index()
obs_per_hybrid = obs_per_hybrid.loc[:, ['Hybrid', 'n']]
obs_per_hybrid['pct'] = obs_per_hybrid.n / obs_per_hybrid.n.sum()
obs_per_hybrid['pct_cumsum'] = np.nan
obs_per_hybrid['random_order'] = 0

obs_per_hybrid

In [ ]:
# fill in the random values to sort on
arr = np.arange(obs_per_hybrid.shape[0])
rng.shuffle(arr)
obs_per_hybrid.random_order = arr

obs_per_hybrid = obs_per_hybrid.sort_values('random_order')
obs_per_hybrid['pct_cumsum'] = obs_per_hybrid.pct.cumsum()
obs_per_hybrid

In [ ]:
# Convert back into phno indices
train_hybrids = list(obs_per_hybrid.loc[(obs_per_hybrid.pct_cumsum <= pct_cumsum_thresh), 'Hybrid'])
test_hybrids  = list(obs_per_hybrid.loc[(obs_per_hybrid.pct_cumsum >  pct_cumsum_thresh), 'Hybrid'])

train_idx = phno.loc[(phno.Hybrid.isin(train_hybrids)), ].index
test_idx  = phno.loc[(phno.Hybrid.isin(test_hybrids)), ].index

In [ ]:
[len(train_idx), len(test_idx)]

In [ ]:
# confirm all observation idxs are have genomic information
assert [] == [e for e in list(train_idx)+list(test_idx) if e not in obs_geno_lookup[:, 0]]

In [ ]:
YMat_cs = calc_cs(YMat[train_idx])
y_cs = apply_cs(YMat, YMat_cs)

In [ ]:
# training_dataloader = DataLoader(
#     ACGTDataset(y = torch.from_numpy(y_cs[train_idx])[:, None].to(torch.float), 
#                 G = torch.from_numpy(ACGT_hilb).to(torch.float), 
#                 idx_original = torch.from_numpy(np.array(train_idx)),
#                 idx_lookup   = torch.from_numpy(np.asarray(obs_geno_lookup)),
#                 use_gpu_num = 0,
#                 device = 'cuda'
#                ),
#     batch_size = 50,
#     shuffle = True
# )

In [ ]:
# testing_dataloader = DataLoader(
#     ACGTDataset(y = torch.from_numpy(y_cs[test_idx])[:, None].to(torch.float), 
#                 G = torch.from_numpy(ACGT_hilb).to(torch.float), 
#                 idx_original = torch.from_numpy(np.array(test_idx)),
#                 idx_lookup   = torch.from_numpy(np.asarray(obs_geno_lookup)),
#                 use_gpu_num = 0,
#                 device = 'cuda'
#                ),
#     batch_size = 50,
#     shuffle = True
# )

In [ ]:
# next(iter(training_dataloader))[0].shape

In [ ]:
# class NeuralNetwork(nn.Module):
#     def __init__(self):
#         super(NeuralNetwork, self).__init__()    

#         def Linear_block(in_size, out_size, drop_pr):
#             block = nn.Sequential(
#                 nn.Linear(in_size, out_size),
#                 nn.ReLU(),
#                 nn.Dropout(drop_pr)
#             )
#             return(block)         
        
        
# #         def Conv1D_Max_block(in_channels, out_channels, kernel_size, stride):
# #             block = nn.Sequential(
# #                 nn.Conv1d(
# #                     in_channels= in_channels, # second channel
# #                     out_channels= out_channels,
# #                     kernel_size= kernel_size,
# #                     stride= stride
# #                 ), 
# #                 nn.MaxPool1d((kernel_size,), stride=stride)
# #             )
# #             return(block)
        
#         self.x_network = nn.Sequential(
#             nn.Conv2d(
#                     in_channels= 4, 
#                     out_channels= 4,
#                     kernel_size= (3, 3),
#                     stride= 2,
#                     padding = 1,
#                     bias = True
#                 ),
#             nn.Conv2d(
#                     in_channels= 4, 
#                     out_channels= 4,
#                     kernel_size= (3, 3),
#                     stride= 2,
#                     padding = 1,
#                     bias = True
#                 ),
#             nn.Conv2d(
#                     in_channels= 4, 
#                     out_channels= 4,
#                     kernel_size= (3, 3),
#                     stride= 2,
#                     padding = 1,
#                     bias = True
#                 ),
#             nn.Conv2d(
#                     in_channels= 4, 
#                     out_channels= 4,
#                     kernel_size= (3, 3),
#                     stride= 2,
#                     padding = 1,
#                     bias = True
#                 ),
#             nn.Conv2d(
#                     in_channels= 4, 
#                     out_channels= 4,
#                     kernel_size= (3, 3),
#                     stride= 2,
#                     padding = 1,
#                     bias = True
#                 ), 
#             nn.Flatten(),            
# #             Linear_block(in_size = 116, out_size = 32, drop_pr = 0.3),
#             nn.Linear(512, 1)
#         )
        
#     def forward(self, x):
#         x_out = self.x_network(x)
#         return x_out

# model = NeuralNetwork().to(device)

# model(next(iter(training_dataloader))[0]).shape

# # torch.Size([50, 4, 256, 512])

In [ ]:
# # small scale test
# model, loss_df = train_nn(
#     cache_path,
#     training_dataloader,
#     testing_dataloader,
#     model,
#     learning_rate = 1e-3,
#     batch_size = 50, #dataloader_batch_size,
#     epochs = 1 #(run_epochs - epochs_run)
# )

# loss_df

In [ ]:
# estimate_iterations(sec_per_it = 95)

In [ ]:
# remove_matching_files(
#     cache_path,
#     match_regex_list = ['model\.pt'],
#     dry_run = False
# )

In [ ]:
# dataloader_batch_size = 50
# run_epochs = 100

# # don't run if either of these exist because there may be cases where we want the results but not the model
# import re

# if not os.path.exists(cache_path+'/model.pt'): 
#     # Shared setup (train from scratch and load latest)
#     model = NeuralNetwork()

#     # find the biggest model to save
#     saved_models = os.listdir(cache_path)
#     saved_models = [e for e in saved_models if re.match('model*', e)]

#     if saved_models == []:
#         epochs_run = 0
#     else:
#         saved_models = [e for e in saved_models if e != 'model.pt']
#         # if there are saved models reload and resume training
#         saved_models_numbers = [int(e.replace('model_', ''
#                                     ).replace('.pt', ''
#                                     ).split('_')[0]) for e in saved_models]
#         # saved_models
#         epochs_run = max(saved_models_numbers)+1 # add 1 to account for 0 index
#         latest_model = [e for e in saved_models if re.match(
#             '^model_'+str(epochs_run-1)+'_.*\.pt$', e)][0] # subtract 1 to convert back
#         model.load_state_dict(torch.load(cache_path+latest_model))
#         print('Resuming Training: '+str(epochs_run)+'/'+str(run_epochs)+' epochs run.')
    
#     model.to(device)    

#     model, loss_df = train_nn(
#         cache_path,
#         training_dataloader,
#         testing_dataloader,
#         model,
#         learning_rate = 1e-3,
#         batch_size = dataloader_batch_size,
#         epochs = (run_epochs - epochs_run)
#     )
    
#     # experimental outputs:
#     # 1. Model
#     torch.save(model.state_dict(), cache_path+'/model.pt') # convention is to use .pt or .pth

#     # 2. loss_df    
#     # If this is resuming training, load and extend the existing loss dataframe
#     if os.path.exists(cache_path+'/loss_df.csv'):
#         loss_df_on_disk = pd.read_csv(cache_path+'/loss_df.csv')
#         epoch_offset = 1 + loss_df_on_disk['Epoch'].max()
#         loss_df['Epoch'] = loss_df['Epoch'] + epoch_offset
#         loss_df = pd.concat([loss_df_on_disk, loss_df])
#     loss_df.to_csv(cache_path+'/loss_df.csv', index=False)  
    
#     # 3. predictions 
#     yhats = pd.concat([
#         yhat_loop(testing_dataloader, model).assign(Split = 'Test'),
#         yhat_loop(training_dataloader, model).assign(Split = 'Train')], axis = 0)

#     yhats.to_csv(cache_path+'/yhats.csv', index=False)

### Standard Visualizations

In [ ]:
# scale_dict = {'y1':YMat_cs}
# import plotly.graph_objects as go

In [ ]:
# naieve_yhat = training_dataloader.dataset.y.mean()

# naieve_MSE_Train = reverse_cs( 
#     np.array(((naieve_yhat - training_dataloader.dataset.y)**2)).mean(),
#     scale_dict['y1']
# )

# naieve_MSE_Test = reverse_cs( 
#     np.array(((naieve_yhat - testing_dataloader.dataset.y)**2)).mean(),
#     scale_dict['y1']
# )

# naieve_MSE_Train, naieve_MSE_Test



# loss_df = pd.read_csv(cache_path+'/loss_df.csv')

# loss_df.TrainMSE = reverse_cs(loss_df.TrainMSE, scale_dict['y1'])
# loss_df.TestMSE  = reverse_cs(loss_df.TestMSE , scale_dict['y1'])


# fig = go.Figure()
# fig.add_trace(go.Scatter(x=loss_df.Epoch, y=loss_df.TestMSE,
#                     mode='lines', name='Test'))
# fig.add_trace(go.Scatter(x=loss_df.Epoch, y=loss_df.TrainMSE,
#                     mode='lines', name='Train'))

# fig.add_trace(go.Scatter(x=loss_df.Epoch, y=[naieve_MSE_Test  for e in range(len(loss_df.Epoch))], 
#                          mode='lines', name='Naieve Test'))
# fig.add_trace(go.Scatter(x=loss_df.Epoch, y=[naieve_MSE_Train for e in range(len(loss_df.Epoch))], 
#                          mode='lines', name='Naieve Train'))
# fig.show()

In [ ]:
# yhats = pd.read_csv(cache_path+'/yhats.csv')

# # px.scatter(yhats, x = 'y_true', y = 'y_pred', color = 'Split')

In [ ]:
# yhats.y_true = reverse_cs(yhats.y_true, scale_dict['y1'])
# yhats.y_pred = reverse_cs(yhats.y_pred, scale_dict['y1'])

# # px.scatter(yhats, x = 'y_true', y = 'y_pred', color = 'Split', trendline="ols")

In [ ]:
# yhats['Error'] = yhats.y_pred - yhats.y_true

# px.histogram(yhats, x = 'Error', color = 'Split',
#              marginal="box", # can be `rug`, `violin`
#              nbins= 50)